In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 


In [2]:

# Charger le DataFrame (par exemple, depuis un fichier Excel ou CSV)
df = pd.read_excel('Double.xlsx')  # Remplacez par votre fichier

# Afficher les valeurs uniques des colonnes level et grade 
valeurs_uniques = df['Grade'].unique()
valeurs_uniques1 = df['Level'].unique()
print(valeurs_uniques)
print (valeurs_uniques1)

df
df.columns

[2 3 0 1]
[5 2 3 4 0 1 6]


Index(['MatchId', 'WeekID', 'TournamentId', 'CategoryID', 'Grade', 'Level',
       'Category Name ', 'Date', 'EventId', 'Event', 'Round', 'Pair1Id',
       'Player1IDP1', 'Player2IDP1', 'Pair2Id', 'Player1IDP2', 'Player2IDP2',
       'WinnerId', 'LastName', 'FirstName', 'Duration', 'ScoreP1', 'ScoreP2',
       'ScoreP1Set1', 'ScoreP2Set1', 'ScoreP1Set2', 'ScoreP2Set2',
       'ScoreP1Set3', 'ScoreP2Set3', 'points P1', 'points P2', 'Total',
       'RetiredId', 'WalkoverId'],
      dtype='object')

In [3]:
# Dictionnaire des multiplicateurs que j'ai créer seule , retrouver le tableau sur les annexes 
multiplicateurs = {
    (0, 0): 0.5, (0, 1): 0.6, (0, 2): 0.7, (0, 3): 0.8, (0, 4): 0.9, (0, 5): 1.0, (0, 6): 1.1,
    (1, 0): 1.2, (1, 1): 1.3, (1, 2): 1.4, (1, 3): 1.5, (1, 4): 1.6, (1, 5): 1.7, (1, 6): 1.8,
    (2, 0): 1.9, (2, 1): 2.0, (2, 2): 2.2, (2, 3): 2.4, (2, 4): 2.6, (2, 5): 2.8, (2, 6): 3.0,
    (3, 0): 3.2, (3, 1): 3.5, (3, 2): 3.8, (3, 3): 4.0, (3, 4): 4.3, (3, 5): 4.6, (3, 6): 5.0
}


# Fonction pour obtenir le multiplicateur basé sur 'level' et 'grade'
def get_multiplicateur(row):
    return multiplicateurs.get((row['Grade'], row['Level']), 1.0)  # Valeur par défaut 1.0 si pas trouvé

# Appliquer la fonction pour créer la colonne 'multiplicateur'
df['multiplicateurs'] = df.apply(get_multiplicateur, axis=1)

# Afficher le DataFrame mis à jour
print(df['multiplicateurs'])

0        2.8
1        2.8
2        2.8
3        2.8
4        2.8
        ... 
40781    0.5
40782    0.5
40783    0.5
40784    0.5
40785    0.5
Name: multiplicateurs, Length: 40786, dtype: float64


In [4]:
# Création d'un DataFrame vide avec les colonnes 'grade', 'level' et 'multiplicateur'
data = []

# Ajouter toutes les combinaisons de grade et level avec leur multiplicateur pour une meilleure visualisation
for grade in range(4):
    for level in range(7):
        data.append([grade, level, multiplicateurs.get((grade, level))])

# Créer le DataFrame
dff = pd.DataFrame(data, columns=['Grade', 'Level', 'multiplicateurs'])

# Affichage du DataFrame
print(dff)

print(df.columns)

    Grade  Level  multiplicateurs
0       0      0              0.5
1       0      1              0.6
2       0      2              0.7
3       0      3              0.8
4       0      4              0.9
5       0      5              1.0
6       0      6              1.1
7       1      0              1.2
8       1      1              1.3
9       1      2              1.4
10      1      3              1.5
11      1      4              1.6
12      1      5              1.7
13      1      6              1.8
14      2      0              1.9
15      2      1              2.0
16      2      2              2.2
17      2      3              2.4
18      2      4              2.6
19      2      5              2.8
20      2      6              3.0
21      3      0              3.2
22      3      1              3.5
23      3      2              3.8
24      3      3              4.0
25      3      4              4.3
26      3      5              4.6
27      3      6              5.0
Index(['MatchI

In [5]:
print(df.columns)

Index(['MatchId', 'WeekID', 'TournamentId', 'CategoryID', 'Grade', 'Level',
       'Category Name ', 'Date', 'EventId', 'Event', 'Round', 'Pair1Id',
       'Player1IDP1', 'Player2IDP1', 'Pair2Id', 'Player1IDP2', 'Player2IDP2',
       'WinnerId', 'LastName', 'FirstName', 'Duration', 'ScoreP1', 'ScoreP2',
       'ScoreP1Set1', 'ScoreP2Set1', 'ScoreP1Set2', 'ScoreP2Set2',
       'ScoreP1Set3', 'ScoreP2Set3', 'points P1', 'points P2', 'Total',
       'RetiredId', 'WalkoverId', 'multiplicateurs'],
      dtype='object')


In [6]:
# Dictionnaire de mappage pour encoder les valeurs de la colonne 'Round'
mapping = {
    'Qualification round of 16': 0,
    'Qualification quarter final': 0,
    'Qualification round of 64': 0,
    'Qualification round of 32': 0,
    'Qualification round of 128': 0,
    'Round of 256': 1,
    'Round of 128': 2,
    'Round of 64': 3,
    'Round of 32': 4,
    'Round of 16': 5,
    'Quarter final': 6,
    'Semi final': 7,
    'Final': 8,
    '1': 9,
    '2': 9,
    '3': 9,
    'Round 1': 9,
    'Round 2': 9,
    'Round 3': 9,
    'Round 4': 9,
    'Round 5': 9,
    '3rd/4th place': 10,
    None: -1,  # Pour les valeurs manquantes (nan)
    float('nan'): -1  # Encodage pour les NaN si pandas les traite explicitement
}

# Encodage de la colonne 'Round'
df['Encoded Round'] = df['Round'].map(mapping)

# Vérification des résultats
print(df[['Round', 'Encoded Round']].head(20))



# Encodage de la durée en catégories
def encode_duration(Duration):
    if Duration < 25:
        return 0
    elif 25 <= Duration <= 35:
        return 1
    elif 35 < Duration <= 45:
        return 2
    else:
        return 3

# Ajouter une colonne encodée pour Duration
df['Encoded Duration'] = df['Duration'].apply(encode_duration)

# Vérification des résultats
print(df[['Duration', 'Encoded Duration']])



# Étape 1 : Compter le nombre de victoires cumulées pour chaque joueur
victoires_par_joueur = df['WinnerId'].value_counts()

# Étape 2 : Associer les victoires cumulées à chaque joueur
df['Victoires Cumulées P1'] = df['Player1IDP1'].map(victoires_par_joueur).fillna(0).astype(int)
df['Victoires Cumulées P2'] = df['Player2IDP1'].map(victoires_par_joueur).fillna(0).astype(int)
df['Victoires Cumulées P2'] = df['Player1IDP2'].map(victoires_par_joueur).fillna(0).astype(int)
df['Victoires Cumulées P2'] = df['Player2IDP2'].map(victoires_par_joueur).fillna(0).astype(int)

# Étape 3 : Fonction pour classer les joueurs
def classer_joueur(victoires):
    if victoires <= 3:
        return 0  # Pas bon
    elif 4 <= victoires <= 6:
        return 1  # Moyennement bon
    elif 7 <= victoires <= 10:
        return 2  # Bon
    else:
        return 3  # Très bon

# Ajouter les colonnes de classement
df['Classement P1'] = df['Victoires Cumulées P1'].apply(classer_joueur)
df['Classement P2'] = df['Victoires Cumulées P2'].apply(classer_joueur)







df['Base Points'] = df['multiplicateurs'] * 10
df['PRG P1'] = df ['points P1'] / df ['Total']
df['PRG P2'] = df ['points P2'] / df ['Total']

                          Round  Encoded Round
0   Qualification quarter final            0.0
1   Qualification quarter final            0.0
2   Qualification quarter final            0.0
3   Qualification quarter final            0.0
4                   Round of 32            4.0
5                   Round of 32            4.0
6                   Round of 32            4.0
7                   Round of 32            4.0
8                   Round of 32            4.0
9                   Round of 32            4.0
10                  Round of 32            4.0
11                  Round of 32            4.0
12                  Round of 32            4.0
13                  Round of 32            4.0
14                  Round of 32            4.0
15                  Round of 32            4.0
16                  Round of 32            4.0
17                  Round of 32            4.0
18                  Round of 32            4.0
19                  Round of 32            4.0
       Durati

In [9]:
# Initialisation du score de base pour tous les joueurs 
scores = {player: 1000 for player in set(df['Player1IDP1']).union(set(df['Player2IDP1'])).union(set(df['Player1IDP2'])).union(set(df['Player2IDP2']))}

# Fonction pour calculer le coefficient adversaire
def coef_adversaire(winner_class, loser_class):
    if winner_class > loser_class:
        return 1.1
    elif winner_class < loser_class:
        return 0.9
    else:
        return 1.0

# Fonction pour calculer le score pour chaque joueur
def calculer_nouveau_score(row):
    global scores  # Utiliser le dictionnaire global des scores
    # Paires et joueurs
    pair1_player1 = row['Player1IDP1']
    pair1_player2 = row['Player2IDP1']
    pair2_player1 = row['Player1IDP2']
    pair2_player2 = row['Player2IDP2']
    winner_pair = row['WinnerId']
    
    # Ancien scores
    score_pair1_player1 = scores.get(pair1_player1, 1000)
    score_pair1_player2 = scores.get(pair1_player2, 1000)
    score_pair2_player1 = scores.get(pair2_player1, 1000)
    score_pair2_player2 = scores.get(pair2_player2, 1000)
    
    # Points calculés
    base_points = row['Base Points']
    points_tour = row['Encoded Round']
    points_temps = 1 + row['Encoded Duration'] 
    prg_p1 = row['PRG P1']
    prg_p2 = row['PRG P2']
    
    # Coefficients adversaire
    coef_pair1_player1 = coef_adversaire(row['Classement P1'], row['Classement P2'])
    coef_pair1_player2 = coef_adversaire(row['Classement P1'], row['Classement P2'])
    coef_pair2_player1 = coef_adversaire(row['Classement P2'], row['Classement P1'])
    coef_pair2_player2 = coef_adversaire(row['Classement P2'], row['Classement P1'])
    
    # Calcul du score par joueur
    if winner_pair == row['Pair1Id']:
        # Pair1 gagne
        nouveau_score_pair1_player1 = score_pair1_player1 + (base_points + points_tour) * prg_p1 * points_temps * coef_pair1_player1
        nouveau_score_pair1_player2 = score_pair1_player2 + (base_points + points_tour) * prg_p1 * points_temps * coef_pair1_player2
        nouveau_score_pair2_player1 = score_pair2_player1 - (base_points + points_tour) * prg_p2 * points_temps * coef_pair2_player1
        nouveau_score_pair2_player2 = score_pair2_player2 - (base_points + points_tour) * prg_p2 * points_temps * coef_pair2_player2
    elif winner_pair == row['Pair2Id']:
        # Pair2 gagne
        nouveau_score_pair2_player1 = score_pair2_player1 + (base_points + points_tour) * prg_p2 * points_temps * coef_pair2_player1
        nouveau_score_pair2_player2 = score_pair2_player2 + (base_points + points_tour) * prg_p2 * points_temps * coef_pair2_player2
        nouveau_score_pair1_player1 = score_pair1_player1 - (base_points + points_tour) * prg_p1 * points_temps * coef_pair1_player1
        nouveau_score_pair1_player2 = score_pair1_player2 - (base_points + points_tour) * prg_p1 * points_temps * coef_pair1_player2
    else:
        # Pas de changement
        nouveau_score_pair1_player1, nouveau_score_pair1_player2 = score_pair1_player1, score_pair1_player2
        nouveau_score_pair2_player1, nouveau_score_pair2_player2 = score_pair2_player1, score_pair2_player2
    
    # Mise à jour des scores globaux pour chaque joueur
    scores[pair1_player1] = max(0, nouveau_score_pair1_player1)
    scores[pair1_player2] = max(0, nouveau_score_pair1_player2)
    scores[pair2_player1] = max(0, nouveau_score_pair2_player1)
    scores[pair2_player2] = max(0, nouveau_score_pair2_player2)
    
    # Retourner les anciens scores avec les nouveaux scores
    return (score_pair1_player1, score_pair1_player2, score_pair2_player1, score_pair2_player2, 
            nouveau_score_pair1_player1, nouveau_score_pair1_player2, 
            nouveau_score_pair2_player1, nouveau_score_pair2_player2)

# Appliquer la fonction pour chaque match
df[['Ancien Score P1_1', 'Ancien Score P1_2', 'Ancien Score P2_1', 'Ancien Score P2_2', 
    'Nouveau Score P1_1', 'Nouveau Score P1_2', 'Nouveau Score P2_1', 'Nouveau Score P2_2']] = df.apply(calculer_nouveau_score, axis=1, result_type='expand')

# Affichage des résultats
print(df[['Pair1Id', 'Pair2Id', 'WinnerId', 'Ancien Score P1_1', 'Ancien Score P1_2', 'Ancien Score P2_1', 'Ancien Score P2_2', 
          'Nouveau Score P1_1', 'Nouveau Score P1_2', 'Nouveau Score P2_1', 'Nouveau Score P2_2']].head(10))






   Pair1Id  Pair2Id  WinnerId  Ancien Score P1_1  Ancien Score P1_2  \
0    19875    19868     19875             1000.0             1000.0   
1    21730    19870     21730             1000.0             1000.0   
2    19904    21425     21425             1000.0             1000.0   
3    20007    19864     19864             1000.0             1000.0   
4    19847    19804     19804             1000.0             1000.0   
5    19817    19855     19817             1000.0             1000.0   
6    19837    19795     19837             1000.0             1000.0   
7    19912    21425     21425             1000.0             1000.0   
8    19799    19864     19799             1000.0             1000.0   
9    19953    19906     19953             1000.0             1000.0   

   Ancien Score P2_1  Ancien Score P2_2  Nouveau Score P1_1  \
0        1000.000000        1000.000000         1033.600000   
1        1000.000000        1000.000000         1035.636364   
2        1000.000000        1

In [11]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd



# Exemple de création de la colonne cible "Y" (Gagnant = 1, Perdant = 0)
df['Y'] = (df['WinnerId'] == df['Pair1Id']).astype(int)

X = df[[ 
        'Base Points', 'Encoded Round', 'Encoded Duration','Ancien Score P1_1', 'Ancien Score P1_2']]

X = X.fillna(0)

# Diviser les données en ensembles d'entraînement et de test (80% entraînement, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, df['Y'], test_size=0.2, random_state=42)

# Initialiser le modèle de régression logistique
model = LogisticRegression(max_iter=1000)

# Entraîner le modèle
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Calculer l'accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy du modèle : {accuracy * 100:.2f}%")

# Afficher les coefficients du modèle pour évaluer l'importance des caractéristiques
print("Coefficients du modèle : ", model.coef_)



Accuracy du modèle : 50.01%
Coefficients du modèle :  [[-2.31907004e-03 -2.66174459e-02 -2.02127688e-02  5.93423245e-05
   9.08297782e-05]]
